<a href="https://colab.research.google.com/github/fedhere/DSPS_FBianco/blob/master/HW5/hogg10_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pylab as pl
from scipy.optimize import curve_fit
%pylab inline

In [ ]:
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

def line(x, slope, intercept):
    return x * slope + intercept

def confidence_ellipse(x, y, sx, sy, cov,
                       ax, edgecolor='k', **kwargs):
    
    n = len(x)
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

  
    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    rots = np.arctan2(sy/y, sx/x)
    #print(rots)
    for i in range(n):
        ellipse = Ellipse((0, 0),
        width=sx[i],
        height=sy[i],
        edgecolor=edgecolor,
        facecolor='None',
        **kwargs)
        
        transf = transforms.Affine2D().rotate_deg(-cov[i][1,0]*180/np.pi)\
        .translate(x[i], y[i])
        #.rotate(rots[i])\
        
        ellipse.set_transform(transf + ax.transData)
        ax.add_patch(ellipse)


# read in data

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/fedhere/DSPS_FBianco/master/HW5/hbl2017.csv")
data.head()

In [ ]:
data.rename(columns={"σx":"sx",
             "σy":"sy",
             "ρxy":"rxy"}, inplace=True)

In [ ]:
data.rxy.values

In [ ]:
nall = len(data)
n = nall - 5

In [ ]:
ax = data.plot(x='x', y="y", yerr="sy", xerr="sx", kind="scatter");
data[5:].plot(x='x', y="y", yerr="sy", xerr="sx", kind="scatter", 
              ax=ax, color='r');

function definition

# numpy implementation with curvefit 

### on partial dataset

In [ ]:
rdata = data[4:] 

# linear algebra solution

convert arrays to matrices

In [ ]:
def makematrix(x, y, sx, sy, npar=2, verbose=False):
    if npar == 2:
        a = np.matrix(np.array([ones_like(x), x])).T
    elif npar == 3:
        a = np.matrix(np.array([ones_like(x), x, x * x])).T
    else: 
        print("wrong number of parameters {}".format(npar) + 
                "only linear npar=2) and quadratic (npar=3) fits enabled")
        return np.nan, np.nan, np.nan 
    Y = np.matrix(np.array([y])).T
    c = np.matrix(np.diag(sy**2))
    if verbose:
        #print(Y, a, c)
        print(Y.shape, a.shape, c.shape)
    return a, Y, c

### matrix solution line fit to partial dataset

## 1. partial set linear fit

In [ ]:
a, Y, c = makematrix(rdata.x.values, rdata.y.values, 
                     rdata.sx.values, rdata.sy.values)
bm = np.linalg.inv(a.T.dot(np.linalg.inv(c)).dot(a)).dot(
    a.T.dot(np.linalg.inv(c)).dot(Y))
bbmm = np.array(bm).T[0][::-1]
print("coefficients: slope {:.2f} intercept {:.1f}".format(*bbmm))

In [ ]:
# fit with LA
yhat = a.dot(bm)

In [ ]:
#residuals & error
yres = yhat - Y
s = yres.T.dot(yres) / (n - len(bm[1]))
print(" error from residuals", float(s))

In [ ]:
newx = [data.x.min(), data.x.max()]
ax = figure(figsize=(10,10)).add_subplot();
ax = data.plot(x="x", y="y", yerr="sy", xerr="sx", kind="scatter", ax=ax);
plot(newx, 
     np.vstack(np.array([ones_like(newx), newx])).T.dot(bm), 
     'y--', label='fit: m=%5.3f, b=%5.3f' % tuple(
              (bm[0][0], bm[1][0])));
#LA
plot(rdata.x, np.array(yhat).flatten(), 'r.')
xlim(0,300)
ylim(0,700)
axes().set_aspect(np.diff(xlim()) / np.diff(ylim()));

### uncertainty

## 2. matrix solution line fit to full dataset

In [ ]:
a2, Y2, c2 = makematrix(data.x.values, data.y.values, 
                        data.sx.values, data.sy.values)
bmfull = np.linalg.inv(a2.T.dot(np.linalg.inv(c2)).dot(a2)).dot(
    a2.T.dot(np.linalg.inv(c2)).dot(np.matrix(data.y.values).T))
bmfull

In [ ]:
yhat2 = a2.dot(bmfull)
yres2 = yhat2 - Y2
s2 = yres2.T.dot(yres2) / (nall - 2)
s2

In [ ]:
ax = figure(figsize=(10,10)).add_subplot();
ax = data.plot(x="x", y="y", yerr="sy", xerr="sx", kind="scatter", ax=ax);
plot(newx, 
    np.vstack(np.array([ones_like(newx), newx])).T.dot(bmfull),
     'y--',
          label='fit: m=%5.3f, b=%5.3f' % tuple((bmfull[0][0], bmfull[1][0])));
plot(data.x, np.array(a2.dot(bmfull)).flatten(), 'r.')
xlim(0,300)
ylim(0,700)
axes().set_aspect(np.diff(xlim()) / np.diff(ylim()));


## 5. MCMC

In [ ]:
!pip install emcee

In [ ]:
!pip install corner

In [ ]:
import emcee
import corner
import scipy.optimize as op
from corner import hist2d

In [ ]:
xx = np.arange(data.x.min(), data.x.max(), 0.1)
pl.errorbar(data.x, data.y, data.sy, fmt='.',  color='k')
pl.plot(xx, line(xx, bmfull[1,0], bmfull[0,0]), 'k-', label="full")
pl.plot(xx, line(xx, bm[1,0], bm[0,0]), '-', label="restricted")
pl.xlabel("x")
pl.ylabel("y")
pl.legend();

In [ ]:
def lnlike(theta, x, y, yerr):
    m, b, Yb, Pb, V = theta
    #Yb = 0
    model = m * x + b
    sig2 = yerr**2
    den = 2 * np.pi * sig2
    a = (1 - Pb) / np.sqrt(den) *\
        exp(-(y-model)**2 /  2.0 / sig2)  
    b = 0
    b = Pb / np.sqrt(den + 2*np.pi*V) *\
        exp (-(y - Yb)**2 / 2 / (V + sig2))
    return  np.sum(np.log(a + b))

In [ ]:
def lnprior(theta):
    m, b, Yb, Pb, V = theta
    if -200 < b < 500 and 0 < m < 10.0 :
        #print("1")
        if Pb < 0 or Pb > 1:
            return -np.inf
        #print("2")
        yb = random.randn() * 300
        if  Yb > ymean + yb or Yb < ymean - yb:
            return -np.inf
        #vb = random.randn() * 200
        #if  V > vb + yb or 
        if V < 0:
            return -np.inf
        #print("3")
        return 0.0
    
    return -np.inf

def lnprob(theta, x, y, yerr):
    lp = lnprior(theta)
    #print(lp)
    if not np.isfinite(lp) :
        return -np.inf
    lnl = lnlike(theta, x, y, yerr)
    if  np.isnan(lnl):
        return -np.inf
    return lp + lnl 

In [ ]:
ymean = data.y.mean()
#initialize from the restricted problem LA solution 


In [ ]:
ndim, nwalkers = 5, 100
init = np.hstack([bm[1,0], bm[0,0], ymean, 
                  0.2, 0.1])
pos = [init + 
       init * 1e-4 * np.random.randn(ndim) for i in range(nwalkers)]
sampler = emcee.EnsembleSampler(nwalkers, ndim, 
                                lnprob, 
                                args=(data.x, data.y,
                                     data.sy))
sampler.run_mcmc(pos, 500, progress=True);
for i in range(ndim):
    pl.figure()
    for j in range(100): 
        pl.plot(sampler.chain[j,:,i], 'k', alpha=0.1)
        
samples = sampler.chain[:, 150:, :].reshape((-1, ndim))

In [ ]:
fig = corner.corner(samples, 
                        labels=["$m$", "$b$", "Y", r"$P_b$", "V"],
                        truths=init)
m_mcmc, b_mcmc = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples[:,:2], [16, 50, 84],
                                                axis=0)))

In [ ]:
xl = np.arange(data.x.min() * 0.9, data.x.max() * 1.1, 10)
pl.errorbar(data.x, data.y, yerr=data.sy, fmt=".k")
pl.plot(xx, line(xx, bm[1,0], bm[0,0]), '-', label="LS - restricted")
pl.plot(xl, line(xl, m_mcmc[0], b_mcmc[0]), color="r", lw=2, alpha=0.9, label="likelihood - simple")
for m, b in samples[np.random.randint(len(samples), size=100)][:,:2]:
    pl.plot(xl, m*xl+b, color="k", alpha=0.02)
pl.plot(xl, m*xl+b, color="k", alpha=0.05, label="MCMC")
pl.legend();

# trying all Ps

In [ ]:
x = data.x
y = data.y

In [ ]:
np.random.rand(len(x)) > 0.5

In [ ]:
def lnlikeP(theta, P, x, y, yerr):
    m, b, Yb, V = theta[:4]
    Pb = (P > 0.5).astype(int)
    model = m * x + b
    sig2 = yerr**2
    den = 2 * np.pi * sig2
    a = (1 - Pb) / np.sqrt(den) *\
        exp(-(y-model)**2 /  2.0 / sig2)  
    b = 0
    b = Pb / np.sqrt(den + 2*np.pi*V) *\
        exp (-(y - Yb)**2 / 2 / (V + sig2))
    return  np.sum(np.log(a + b))
lnlikeP((bm[1,0], bm[0,0], 400, 300), np.random.rand(len(x)), 
        x, y, data.sy)

In [ ]:
def lnpriorP(theta, P):
    m, b, Yb, V = theta[:4]
    if -200 < b < 500 and 0 < m < 10.0 :
        #print("1")
        if (P > 1).any() or (P < 0).any():
            return -np.inf
        yb = random.randn() * 300
        if  Yb > ymean + yb or Yb < ymean - yb:
            return -np.inf
        #vb = random.randn() * 200
        #if  V > vb + yb or 
        if V < 0:
            return -np.inf
        #print("3")
        return 0.0
    
    return -np.inf

def lnprobP(theta, x, y, yerr):
    P = theta[4:]
    lp = lnpriorP(theta[:4], P)
    #print(lp)
    if not np.isfinite(lp) :
        return -np.inf
    lnl = lnlikeP(theta[:4], P, x, y, yerr)
    if  np.isnan(lnl):
        return -np.inf
    return lp + lnl #- np.exp((P>0.5).sum())

In [ ]:
probsinit = (np.abs(np.array(a2 * bmfull).flatten() - data.y.values)) / ymean
probsinit

In [ ]:
from multiprocessing import Pool

ndim, nwalkers = 4 + len(x), 300
init = np.hstack([bm[1,0], bm[0,0], ymean, 300,
                  probsinit])
pos = [init + 
       init * 1e-4 * np.random.randn(len(init)) 
       for i in range(nwalkers)]

with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, 
                                lnprobP, 
                                args=(data.x, data.y,
                                     data.sy), pool=pool)
    sampler.run_mcmc(pos, 2000, progress=True);

for i in range(ndim):
    pl.figure()
    for j in range(100): 
        pl.plot(sampler.chain[j,:,i], 'k', alpha=0.1)
        
samples = sampler.chain[:, 150:, :].reshape((-1, ndim))

In [ ]:

m_mcmc, b_mcmc = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples[:,:2], 
                                                [16, 50, 84],
                                                axis=0)))

In [ ]:
probs = samples[:,4:].mean(axis=0) > 0.5
probs_marg = samples[:,4:].mean(axis=0)
probs.sum()

In [ ]:
samples[:,4:].shape

In [ ]:
pl.errorbar(data.x, data.y, data.sy, fmt='.',  color='k')
#for i in range(len(probs)):
#    pl.text(x[i]+10, y[i], probs[i])#"%d"%(samples[:,4+i].mean()))
for i in range(len(probs)):
    if probs_marg[i] < 0.5:
        pl.text(x[i]+20, y[i], "%.1f"%probs_marg[i])#"%d"%(samples[:,4+i].mean()))
        
    else :
         pl.text(x[i]+20, y[i], "%.1f"%probs_marg[i], color='r')#"%d"%(samples[:,4+i].mean()))
       
            #probs[i])
for i in range(len(probs)):
    pl.text(x[i]+10, y[i], "%d,"%i)
pl.plot(xx, line(xx, bm[1,0], bm[0,0]), '-', label="LS - restricted")
pl.plot(xl, line(xl, m_mcmc[0], b_mcmc[0]), color="r", lw=2, alpha=0.9, label="likelihood - Ps\n" + 
        "%.2f x +  %.2f"%(m_mcmc[0], b_mcmc[0]))
pl.xlabel("x")
pl.ylabel("y")
pl.legend();


# New Model with xerr

In [ ]:
cov = np.zeros((len(data), 2, 2))
for i in range(len(data)):
    cov[i] = np.vstack([np.hstack([data.loc[i].sx**2, data.loc[i].rxy]),
                        np.hstack([data.loc[i].rxy, data.loc[i].sy**2])])
#cov = np.vstack(cov)

In [ ]:
def VV(mm) :
    return 1.0 / np.sqrt( 1 + mm**2) * np.vstack([-mm, 1])
    
def DD(bb, mm, x, y, i):
    Di = VV(mm).T.dot(np.vstack([x[i], y[i]]))
    return Di - bb / np.sqrt(1 + mm**2)

def SS(mm, i, cov):
    print(i)
    return VV(mm).T.dot(cov[i]).dot(VV(mm))

def lnlik13(theta, x, y, cov):
    mm, bb = theta
    dd = np.array([VV(mm).T.dot(np.vstack([x[i], y[i]])) -\
        bb / np.sqrt(1 + mm**2) for i in range(len(x))])
    ss = np.array([VV(mm).T.dot(cov[i]).dot(VV(mm)) 
                   for i in range(len(x))])
    return - np.sum(dd**2 / 2 / ss)
    
lnlik13((1,2), data.x.values, data.y.values, cov)


In [ ]:
nll13 = lambda *args: -lnlik13(*args)
result13 = op.minimize(nll13, [bmfull[1,0], bmfull[0,0]], 
                     args=(rdata.x.values, rdata.y.values, cov[4:]))
m13, b13 = result13["x"]
print("slope    : MCMC {:.2f}, LA restricted {:.2f} full {:.2f}".format(
    m13, bm[1,0], bmfull[1,0]))
print("intercept: MCMC {:.2f}, LA restricted {:.2f} full {:.2f}".format(
    b13, bm[0,0], bmfull[0,0]))

In [ ]:
def lnprior13(theta):
    m, b = theta
    if -200 < b < 500 and 0 < m < 10.0 :
        return 0.0    
    return -np.inf

def lnprob13(theta, x, y, c):
    lp = lnprior13(theta)
    #print(lp)
    if not np.isfinite(lp) :
        return -np.inf
    lnl = lnlik13(theta, x, y,  c)
    if  np.isnan(lnl):
        return -np.inf
    return lp + lnl

lnprob13((2, 1), rdata.x.values, rdata.y.values, cov[4:])

In [ ]:
pl.errorbar(rdata.x, rdata.y, rdata.sy, fmt='.',  color='k')
pl.plot(xx, line(xx, bmfull[1,0], bmfull[0,0]), 'k-', label="full")
pl.plot(xx, line(xx, bm[1,0], bm[0,0]), '-', label="restricted")
pl.plot(xx, line(xx, m13, b13), 'r--', label="likelihood xerr yerr")
pl.xlabel("x")
pl.ylabel("y")
pl.legend();


In [ ]:
init = np.hstack([bm[1,0], bm[0,0]])
ndim, nwalkers = len(init), 100
pos = [init + 
       init * 1e-4 * np.random.randn(ndim) 
       for i in range(nwalkers)]
with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, 
                                lnprob13, 
                                args=(rdata.x.values, rdata.y.values,
                                      cov[4:]), pool=pool)

    sampler.run_mcmc(pos, 500, progress=True);
for i in range(ndim):
    pl.figure()
    for j in range(100): 
        pl.plot(sampler.chain[j,:,i], 'k', alpha=0.1)
        
samples = sampler.chain[:, 150:, :].reshape((-1, ndim))



In [ ]:
fig = corner.corner(samples, 
                        labels=["$m$", "$b$"],
                        truths=init)
m_mcmc, b_mcmc = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples[:,:2], [16, 50, 84],
                                                axis=0)))
m_mcmc, b_mcmc, bmfull[1,0], bmfull[0,0]

In [ ]:
init = np.hstack([bmfull[1,0], bmfull[0,0]])
ndim, nwalkers = len(init), 100
pos = [init + 
       init * 1e-4 * np.random.randn(ndim) 
       for i in range(nwalkers)]
with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, 
                                lnprob13, 
                                args=(data.x.values, data.y.values,
                                      cov), pool=pool)

    sampler.run_mcmc(pos, 500, progress=True);
for i in range(ndim):
    pl.figure()
    for j in range(100): 
        pl.plot(sampler.chain[j,:,i], 'k', alpha=0.1)
        
samples = sampler.chain[:, 150:, :].reshape((-1, ndim))



In [ ]:
fig = corner.corner(samples, 
                        labels=["$m$", "$b$"],
                        truths=init)
m_mcmc, b_mcmc = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples[:,:2], [16, 50, 84],
                                                axis=0)))
m_mcmc, b_mcmc, bmfull[1,0], bmfull[0,0]


In [ ]:
xl = np.arange(data.x.min() * 0.9, data.x.max() * 1.1, 10)
pl.errorbar(data.x, data.y, 
            xerr=data.sx, yerr=data.sy, fmt=".k")
pl.plot(xx, line(xx, bm[1,0], bm[0,0]), '-', 
        label="LS - restricted")
pl.plot(xl, line(xl, m_mcmc[0], b_mcmc[0]), 
        color="r", lw=2, alpha=0.9, label="likelihood - simple")
for m, b in samples[np.random.randint(len(samples), 
                                      size=100)][:,:2]:
    pl.plot(xl, m*xl+b, color="k", alpha=0.02)
pl.plot(xl, m*xl+b, color="k", alpha=0.05, label="MCMC")
pl.legend();


# full model w xerr

In [ ]:
def lnlik14(theta, x, y, yerr, cov):
    mm, bb, Yb, Pb, V = theta
    dd = np.array([VV(mm).T.dot(np.vstack([x[i], y[i]])) -  
                   bb * VV(mm)[1] for i in range(len(x))])
    sig2 = yerr**2
    b = exp (-(y - Yb)**2 / 2 / (V + sig2))
    ss = np.array([VV(mm).T.dot(cov[i]).dot(VV(mm)) 
                   for i in range(len(x))])
    
    lnl1 =  (-dd**2 / 2 / ss).flatten()
    ret =  ((1 - Pb) * np.exp(lnl1) +\
      (Pb / np.sqrt(2*np.pi*(V+sig2))) * b)
   
    return np.sum(np.log(ret))

lnlik14((2, 1, 400, 0, 1), data.x.values, data.y.values, 
        data.sy.values, cov)


In [ ]:
def lnprior14(theta):
    m, b, Yb, Pb, V = theta
    if -200 < b < 500 and 0 < m < 15.0 :
        #print("b,m limit")
        if Pb < 0 or Pb > 1:
            #print("P limit")
            return -np.inf
            #print("2")
        yb = random.randn() * 100
        if  Yb > ymean + yb or Yb < ymean - yb:
            #print("Y limit")
            return -np.inf
        #vb = random.randn() * 200
        #if  V > vb + yb or 
        if V < 10:
            #print("V limit")
            return -np.inf
        return 0.0    
    return -np.inf


def lnprob14(theta, x, y, yerr, c):
    lp = lnprior14(theta)
    #print(lp)
    if not np.isfinite(lp) :
        return -np.inf
    lnl = lnlik14(theta, x, y, yerr, c)
    if  np.isnan(lnl):
        #print("nan lnl")
        return -np.inf
    return lp + lnl

In [ ]:
init = np.hstack([bmfull[1,0], bmfull[0,0], ymean, 0.2, 100])
ndim, nwalkers = len(init), 100
pos = [init + 
       init * 1e-4 * np.random.randn(ndim) for i in range(nwalkers)]
with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, 
                                lnprob14, 
                                args=(data.x, data.y, 
                                      data.sy, cov))

    sampler.run_mcmc(pos, 500, progress=True);

In [ ]:
for i in range(ndim):
    pl.figure()
    for j in range(100): 
        pl.plot(sampler.chain[j,:,i], 'k', alpha=0.1)
        
samples = sampler.chain[:, 150:, :].reshape((-1, ndim))

In [ ]:
fig = corner.corner(samples, 
                        labels=["$m$", "$b$", "Y", r"$P_b$", "V"],
                        truths=init)
m_mcmc, b_mcmc = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples[:,:2], [16, 50, 84],
                                                axis=0)))

In [ ]:
xl = np.arange(data.x.min() * 0.9, data.x.max() * 1.1, 10)
pl.errorbar(data.x, data.y, 
            xerr=data.sx, yerr=data.sy, fmt=".k")
pl.plot(xx, line(xx, bm[1,0], bm[0,0]), '-', 
        label="LS - restricted")
pl.plot(xl, line(xl, m_mcmc[0], b_mcmc[0]), 
        color="r", lw=2, alpha=0.9, label="likelihood - simple")
for m, b in samples[np.random.randint(len(samples), 
                                      size=100)][:,:2]:
    pl.plot(xl, m*xl+b, color="k", alpha=0.02)
pl.plot(xl, m*xl+b, color="k", alpha=0.05, label="MCMC")
pl.legend();


In [ ]:
def lnlik15(theta, P, x, y, yerr, cov):
    mm, bb, Yb, V = theta[:4]
    Pb = (P > 0.5).astype(int)
    
    dd = np.array([VV(mm).T.dot(np.vstack([x[i], y[i]])) -  bb*VV(mm)[1]
                  for i in range(len(x))])
    ss = np.array([VV(mm).T.dot(cov[i]).dot(VV(mm)) 
                   for i in range(len(x))])
    sig2 = yerr**2
    
    lnl1 =  - dd**2 / 2 / ss
    
    #print(Pb)
    b = exp (-(y - Yb)**2 / 2 / (V + sig2))
    #lnl1 = 0
    #Pb=0.5
    #Pb=0
    #b=0
    ret =  ((1 - Pb) * np.exp(lnl1) +\
      (Pb / np.sqrt(2*np.pi*(V+sig2))) * b)
    return np.sum(np.log(ret))

def lnprior15(theta, P):
    #print(P>1)
    m, b, Yb, V = theta[:4]
    if 20 < b < 500 and 0 < m < 10.0 :
        #print("1")
        if (P > 1).any() or (P < 0).any():
            return -np.inf
        yb = random.randn() * 300
        if  Yb > ymean + yb or Yb < ymean - yb:
            return -np.inf
        #vb = random.randn() * 200
        #if  V > vb + yb or 
        if V < 0:
            return -np.inf
        #print("3")
        return 0.0
    
    return -np.inf



def lnprob15(theta, x, y, yerr, c):
    P = theta[4:]
    theta = theta[:4]
    lp = lnprior15(theta, P)
    #print(lp)
    if not np.isfinite(lp) :
        return -np.inf
    lnl = lnlik15(theta, P, x, y, yerr, c)
    if  np.isnan(lnl):
        #print("nan lnl")
        return -np.inf
    return lp + lnl #- np.exp((P>0.5).sum())


In [ ]:
lnlik15(np.hstack([2, 200, ymean, 200]), probsinit, 
        data.x.values, 
        data.y.values, data.sy.values, cov)


In [ ]:
pl.errorbar(data.x, data.y, data.sy, fmt='.',  color='k')
pl.plot(xx, line(xx, bmfull[1,0], bmfull[0,0]), 'k-', label="full")
pl.plot(xx, line(xx, bm[1,0], bm[0,0]), '-', label="restricted")
pl.plot(xx, line(xx, m13, b13), 'r--', label="likelihood xerr yerr")
pl.xlabel("x")
pl.ylabel("y")
pl.legend();

In [ ]:
probsinit 

In [ ]:
init = np.hstack([bm[1,0], bm[0,0], ymean, 200,
                  probsinit])
ndim, nwalkers = len(init), 300
pos = [init + init * 
       1e-4 * np.random.randn(ndim) for i in range(nwalkers)]
with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, 
                                lnprob15, 
                                args=(data.x.values, 
                                      data.y.values, 
                                      data.sy.values, 
                                      cov), pool=pool)
    sampler.run_mcmc(pos, 2500, progress=True);
for i in range(ndim):
    pl.figure()
    for j in range(100): 
        pl.plot(sampler.chain[j,:,i], 'k', alpha=0.1)
        
samples = sampler.chain[:, 100:, :].reshape((-1, ndim))

In [ ]:
hist2d(samples[:,0], samples[:,1])

In [ ]:
probs = samples[:,4:].mean(axis=0) > 0.5
probs_marg = samples[:,4:].mean(axis=0) 
probs.sum()

In [ ]:
pl.errorbar(data.x, data.y, 
            xerr=data.sx, yerr=data.sy, 
            fmt='.',  color='k')
#for i in range(len(probs)):
#    pl.text(x[i]+10, y[i], probs[i])#"%d"%(samples[:,4+i].mean()))
for i in range(len(probs)):
    if probs_marg[i] < 0.5:
        pl.text(x[i]+20, y[i], "%.1f"%probs_marg[i])#"%d"%(samples[:,4+i].mean()))
        
    else :
         pl.text(x[i]+20, y[i], "%.1f"%probs_marg[i], color='r')#"%d"%(samples[:,4+i].mean()))
       
            #probs[i])
for i in range(len(probs)):
    pl.text(x[i]+10, y[i], "%d,"%i)
pl.plot(xx, line(xx, bm[1,0], bm[0,0]), '-', label="LS - restricted")
pl.plot(xl, line(xl, m_mcmc[0], b_mcmc[0]), color="r", lw=2, alpha=0.9, label="likelihood - Ps\n" + 
        "%.2f x +  %.2f"%(m_mcmc[0], b_mcmc[0]))
pl.xlabel("x")
pl.ylabel("y")
pl.legend();

In [ ]:
(np.hstack([np.zeros(10), [1]]) > 0).any()

In [ ]:
m_mcmc13, b_mcmc13 = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples[:,:2], [16, 50, 84],
                                                axis=0)))
m_mcmc13, b_mcmc13

In [ ]:
ax = pl.figure().add_subplot(111)
ax.errorbar(rdata.x, rdata.y, 
            #xerr=rdata.sx, yerr=rdata.sy, 
            fmt='.',  color='k')
ax.plot(xx, line(xx, m_mcmc13[0], b_mcmc13[0]), 'k-', 
        label="y={:.2f}x+{:.2f}".format(
                        m_mcmc13[0], b_mcmc13[0]))

confidence_ellipse(rdata.x.values, rdata.y.values, 
                   rdata.sx.values, rdata.sy.values, 
                   cov[4:], ax)
ax.set_xlabel("x")
ax.set_ylabel("y")
pl.legend();

In [ ]:
pos = [np.array([m13, b13]) + 
       np.array([m13, b13]) * 
       1e-4 * np.random.randn(ndim) for i in range(nwalkers)]
sampler13full = emcee.EnsembleSampler(nwalkers, ndim, 
                                lnprob13, 
                                args=(data.x.values, data.y.values, 
                                      cov))
sampler13full.run_mcmc(pos, 500, progress=True);
for i in range(ndim):
    pl.figure()
    for j in range(100): 
        pl.plot(sampler13full.chain[j,:,i], 'k', alpha=0.1)
        
samples13full = sampler13full.chain[:, 100:, :].reshape((-1, ndim))

In [ ]:
m_mcmc13full, b_mcmc13full = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples13full[:,:2], [16, 50, 84],
                                                axis=0)))
ax = pl.figure().add_subplot(111)
ax.errorbar(data.x, data.y, 
            #xerr=rdata.sx, yerr=rdata.sy, 
            fmt='.',  color='k')
ax.plot(xx, line(xx, m_mcmc13full[0], b_mcmc13full[0]), 'k-', 
        label="y={:.2f}x+{:.2f}".format(
                        m_mcmc13full[0], b_mcmc13full[0]))
confidence_ellipse(data.x.values, data.y.values, 
                   data.sx.values, data.sy.values, 
                   cov, ax)
ax.set_xlabel("x")
ax.set_ylabel("y")
pl.legend();

In [ ]:
nll14 = lambda *args: -lnlik14(*args)
result14 = op.minimize(nll14, [bmfull[1,0], bmfull[0,0], ymean, 0, 0.1],
                     args=(rdata.x.values, rdata.y.values,
                           rdata.sy.values, cov[4:]))
m14, b14, tmp, tmp, tmp = result14["x"]
m14, b14

In [ ]:
ndim, nwalkers = len(result14["x"]), 100
#initialize from the restricted problem LA solution 
init = np.hstack([bm[1,0], bm[0,0], ymean, 
                  0.2, 0.1])
pos = [init + 
       init * 1e-4 * np.random.randn(ndim) for i in range(nwalkers)]

sampler14full = emcee.EnsembleSampler(nwalkers, ndim, 
                                lnprob14, 
                                args=(data.x.values, data.y.values,
                                      data.sy.values,
                                      cov))
sampler14full.run_mcmc(pos, 2000, progress=True);
       


In [ ]:
for i in range(ndim):
    pl.figure()
    for j in range(100): 
        pl.plot(sampler14full.chain[j,:,i], 'k', alpha=0.1)

samples14full = sampler14full.chain[:, 100:, :].reshape((-1, ndim))
fig = corner.corner(samples14full, labels=["$m$", "$b$", "Y", r"$P_b$", "V"],
                      truths=init)

In [ ]:
pl.hist(samples14full[:,3]), samples14full[:,3].shape

In [ ]:
sampler14full = emcee.EnsembleSampler(nwalkers, ndim, 
                                lnprob14, 
                                args=(data.x.values, data.y.values,
                                      data.sy.values,
                                      cov))
sampler14full.run_mcmc(pos, 2000);
       

In [ ]:
ndim, nwalkers = len(data.x.values) + 4, 100
#initialize from the restricted problem LA solution 
init = np.hstack([bm[1,0], bm[0,0], ymean, 
                  np.zeros_like(data.x.values) + 0.5, 0.1])
pos = [init + 
       init * 1e-4 * np.random.randn(len(init)) for i in range(nwalkers)]

In [ ]:
                       
sampler15full = emcee.EnsembleSampler(nwalkers, ndim, 
                                lnprob15, 
                                args=(data.x.values, data.y.values,
                                      data.sy.values,
                                      cov))
sampler15full.run_mcmc(pos, 1000);
       

In [ ]:
for i in range(ndim):
    pl.figure()
    for j in range(100): 
        pl.plot(sampler15full.chain[j,:,i], 'k', alpha=0.1)

samples15full = sampler15full.chain[:, 100:, :].reshape((-1, ndim))
fig = corner.corner(samples15full)

In [ ]:
m_mcmc15full, b_mcmc15full = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples15full[:,:2], [16, 50, 84],
                                                axis=0)))
ax = pl.figure().add_subplot(111)
ax.errorbar(data.x, data.y, 
            #xerr=rdata.sx, yerr=rdata.sy, 
            fmt='.',  color='k')
ax.plot(xx, line(xx, m_mcmc15full[0], b_mcmc15full[0]), 'k-', 
        label="y={:.2f}x+{:.2f}".format(
                        m_mcmc15full[0], b_mcmc15full[0]))
confidence_ellipse(data.x.values, data.y.values, 
                   data.sx.values, data.sy.values, 
                   cov, ax)
ax.set_xlabel("x")
ax.set_ylabel("y")
pl.legend();

In [ ]:
probs = [samples15full[:,i].mean()>0.5 for i in range(3,23)]

In [ ]:
len(data.x.values)

In [ ]:
m_mcmc15full, b_mcmc15full = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples15full[:,:2], [16, 50, 84],
                                                axis=0)))
ax = pl.figure().add_subplot(111)
ax.errorbar(data.x, data.y, 
            #xerr=rdata.sx, yerr=rdata.sy, 
            fmt='.',  color='k')
ax.plot(xx, line(xx, m_mcmc15full[0], b_mcmc15full[0]), 'k-', 
        label="y={:.2f}x+{:.2f}".format(
                        m_mcmc15full[0], b_mcmc15full[0]))
confidence_ellipse(data.x.values, data.y.values, 
                   data.sx.values, data.sy.values, 
                   cov, ax)
for i in range(len(probs)):
    pl.text(data.x.values[i], data.y.values[i], probs[i])
ax.set_xlabel("x")
ax.set_ylabel("y")
pl.legend();

In [ ]:
hist2d(samples14full[:,1], samples14full[:,0])
print(m_mcmc15full[0], b_mcmc15full[0], bm, bmfull)